decription is [here](https://www.fi.muni.cz/~xpelanek/publications/jla-dataset.pdf)

https://www.fi.muni.cz/adaptivelearning/data/slepemapy/

In [1]:
import os
import sys
import gzip
import time
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
from utils import *

In [5]:
prefix = '/mnt/qb/work/mlcolab/hzhou52/kt/slepemapy'
answer = pd.read_csv(os.path.join(prefix, 'answer.csv'), sep=';')
answer.head()
# type: Type of	question: 1)find the given place on	the	map;	
# 2) pick the name for the highlighted place

,id,user,place_asked,place_answered,type,inserted,response_time,place_map,language,options,ip_country,ip_id
0,2407,75,131,NaN,2,2013-09-26 14:45:51,8250,NaN,0,"[57, 82, 131, 77]",NaN,NaN
1,2408,75,130,69.0,1,2013-09-26 14:46:03,8526,NaN,0,"[58, 219, 69, 130]",NaN,NaN
2,2409,75,136,128.0,1,2013-09-26 14:46:10,4060,NaN,0,"[128, 136, 216, 54]",NaN,NaN
3,2410,75,126,126.0,1,2013-09-26 14:46:20,1842,NaN,0,"[153, 58, 126, 214]",NaN,NaN
4,2411,75,101,101.0,2,2013-09-26 14:46:33,10258,NaN,0,"[153, 163, 101, 195]",NaN,NaN


In [8]:
check_nan(answer)
count_unique(answer, ['place_asked', 'user'])
print(len(answer))

number of NaN in column id: 0
number of NaN in column user: 0
number of NaN in column place_asked: 0
number of NaN in column place_answered: 281706
number of NaN in column type: 0
number of NaN in column inserted: 0
number of NaN in column response_time: 0
number of NaN in column place_map: 496987
number of NaN in column language: 0
number of NaN in column options: 0
number of NaN in column ip_country: 781171
number of NaN in column ip_id: 773545
number of unique value in place_asked: 1458
number of unique value in user: 91331
10087305


In [10]:
answer = remove_log(answer, 'user')
print(len(answer))
count_unique(answer, ['place_asked', 'user'])

10083975
number of unique value in place_asked: 1458
number of unique value in user: 88001


# pre-process

In [12]:
# create new data
df = pd.DataFrame(columns=['user_id', 'problem_id', 'skill_id', 'timestamp', 'dwell_time', 'correct'])
df.user_id = answer.user
df.problem_id = answer.place_asked
df.skill_id = answer.place_asked
df.timestamp = answer.inserted.apply(lambda t: time.mktime(time.strptime(t, '%Y-%m-%d %H:%M:%S')))
df.dwell_time = answer.response_time.apply(lambda t: t / 1000.0)
df.correct = (answer['place_asked'].astype(float) == answer['place_answered'].astype(float))*1

In [15]:
# re-index
# - skill
skill_id = list(df['skill_id'].unique())
skill_dict = dict(zip(skill_id, range(len(skill_id))))
df['skill_id'] = df['skill_id'].apply(lambda x: skill_dict[x])

# - problem
pro_id = list(df['problem_id'].unique())
pro_dict = dict(zip(pro_id, range(len(pro_id))))
df['problem_id'] = df['problem_id'].apply(lambda x: pro_dict[x])

# - student
stu_id = list(df['user_id'].unique())
stu_dict = dict(zip(stu_id, range(len(stu_id))))
df['user_id'] = df['user_id'].apply(lambda x: stu_dict[x])

In [19]:
# Adujust dtypes
df = df.astype({'correct': np.float64, 'dwell_time': np.float64, 'timestamp': np.float64})
df.dtypes

user_id         int64
problem_id      int64
skill_id        int64
timestamp     float64
dwell_time    float64
correct         int64
dtype: object

In [21]:
# Save
df.to_csv(os.path.join(prefix, 'interactions.csv'), sep='\t', index=False)

In [23]:
count_unique(df, list(df.columns))
len(df)

number of unique value in user_id: 88001
number of unique value in problem_id: 1458
number of unique value in skill_id: 1458
number of unique value in timestamp: 7415199
number of unique value in dwell_time: 123341
number of unique value in correct: 2


10083975